## RetrievalQA chain

Único adendo é que ele não é eficiente ao se trabalhar com memória

!(app\Answers\Retrieval.png)

### Métodos adicionais para responder a pergunta.

!(app\Answers\Additional methods.png)

In [9]:
import os
import glob
from typing import List
from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA

from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
def create_chat(t=0.0):
    """
    Configura os objetos do AzureChatOpenAI para serem utilizados.
 
    Args:
        t (float): O valor da temperatura para determinar o comportamento da resposta, por padrão é 0.0.
   
    Returns:
        AzureChatOpenAI: O objeto de AzureChatOpenAi configurado.
    """
    llm_chat = AzureChatOpenAI(openai_api_base=os.getenv("OPENAI_API_BASE"),
                    openai_api_version=os.getenv("OPENAI_API_VERSION"),
                    openai_api_key=os.getenv("OPENAI_API_KEY"),
                    openai_api_type=os.getenv("OPENAI_API_TYPE"),
                    deployment_name=os.getenv("DEPLOYMENT_NAME"),
                    temperature=t
    )
    return llm_chat

In [3]:
# load pdfs
loaders = [
    PyPDFLoader(r'C:\AI\Cursos\LangChain_Chat_Data\data\Docker_para_desenvolvedores.pdf'),
           PyPDFLoader(r'C:\AI\Cursos\LangChain_Chat_Data\data\Containers_com_Docker.pdf')
        ]   
           
docs = []

for loader in loaders:
    docs.extend(loader.load())

In [4]:
# split

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
)

splits = text_splitter.split_documents(docs)
# Total de chunks
len(splits)

336

In [5]:
def create_embeddings():
    embeddings = OpenAIEmbeddings(
        deployment = os.getenv("EMBEDDING_DEPLOYMENT_NAME"),
    )
    return embeddings

embedding = create_embeddings()

In [6]:
# Armazenamento no VectorDB
vectordb = Chroma.from_documents(
    
    documents=splits,
    embedding=embedding,
    #persist_directory=persist_directory
)

print(vectordb._collection.count())

336


In [7]:
query = "O que é uma imagem e em qual sistema operacional consigo construir uma?"

# Busca por similaridades
docs = vectordb.similarity_search(query,k=3)
len(docs)

3

In [10]:
# Conexão com o modelo
llm = create_chat()

# Conexão para que seja possível responder a pergunta.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever = vectordb.as_retriever(),
)

In [11]:
result = qa_chain({"query": query})
result["result"]

'Uma imagem no Docker é uma abstração da infraestrutura em estado somente leitura, de onde será instanciado o container. É possível construir uma imagem em qualquer sistema operacional que suporte o Docker, como Linux, Windows e macOS. O processo de construção de uma imagem é feito através do Dockerfile, que é um arquivo que aceita rotinas em shell script para serem executadas.'

In [13]:
# Construir o prompt para auxiliar o modelo na forma e nas informações auxiliares que ajudarão a responder as perguntas.
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
prompt = PromptTemplate.from_template(template)


In [15]:
# Rodar a chain

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever = vectordb.as_retriever(),
    return_source_documents=True,
    # Passando a estrutura que irá auxiliar
    chain_type_kwargs={"prompt":prompt}
)

In [17]:
# Fazer a pergunta e retornar o resultado.
query = "Qual a relação entre uma imagem e um container?"
result = qa_chain({"query": query})
result["result"]

'Uma imagem é a abstração da infraestrutura em estado somente leitura, enquanto um container é iniciado a partir de uma imagem e é a instância em execução da imagem. Um container só pode ser iniciado a partir de uma única imagem e, caso seja necessário um comportamento diferente, é necessário customizar a imagem. Thanks for asking!'

In [18]:
# Verificar os documentos originais.
result["source_documents"][0]

Document(page_content='Criando sua própria\nimagem no Docker\nAntes de explicarmos como criar sua imagem, vale a pena\ntocarmos em uma questão que normalmente confunde inici-\nantesdodocker:“Imagemoucontainer?”\nQual a diferença entre Imagem e\nContainer?\nTraçandoumparalelocomoconceitode orientaçãoaobjeto⁴⁴ ,\naimagemé a classe e o container o objeto. A imagem é\na abstração da infraestrutura em estado somente leitura, de\nondeseráinstanciadoocontainer.\nTodo container é iniciado a partir de uma imagem, dessa\nformapodemosconcluirquenuncateremosumaimagemem\nexecução.\nUm container só pode ser iniciado a partir de uma única\nimagem. Caso deseje um comportamento diferente, será ne-\ncessáriocustomizaraimagem.\nAnatomia da imagem\nAsimagenspodemseroficiaisounãooficiais.\n⁴⁴https://pt.wikipedia.org/wiki/Orienta%C3%A7%C3%A3o_a_objetos', metadata={'page': 40, 'source': 'C:\\AI\\Cursos\\LangChain_Chat_Data\\data\\Docker_para_desenvolvedores.pdf'})

In [19]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [20]:
result = qa_chain_mr({"query": query})
result["result"]

'A relação entre uma imagem e um container é que o Docker utiliza a imagem para criar o container durante a etapa de construção. Quando criamos um container, o Docker monta o rootfs em modo read-only e utiliza a imagem para criar uma camada com permissões de read-write sobre a camada de read-only. Em outras palavras, a imagem é a base para a criação do container.'

In [21]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": query})
result["result"]

'The relationship between an image and a container in the context of Docker is that an image is used as the basis for creating a container. During the image creation process, Docker uses a read-only root file system (rootfs) that includes the typical directory structure and all necessary configuration files, binaries, and libraries for program operation. Instead of modifying the rootfs to read-write mode, Docker uses unionfs to create a layer with read-write permissions over the read-only layer. This allows for the creation of multiple containers from the same image, each with its own read-write layer for making changes and storing data. The image can be built using a Dockerfile, which is a script that specifies the base image, any additional software to be installed, and any configuration changes to be made. Once the Dockerfile is created, it can be used to build the image, which can then be used to create containers.'